# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [21]:
# imports
import panel as pn
from panel.interact import interact
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [22]:
# Set up Panel Plotly extension
pn.extension('plotly')

In [23]:
pd.options.plotting.backend = 'holoviews'
import hvplot
import hvplot.pandas

In [24]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [25]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

# Load neighborhoods coordinates data
file_path = Path("neighborhoods_coordinates.csv")
neighborhood_data = pd.read_csv(file_path)
neighborhood_data.head()
# mean number of housing units per year data
file_path = Path("sfo_mean.csv")
sfo_data2 = pd.read_csv(file_path, index_col='year')
sfo_data2.head()

,housing_units,sale_price_sqr_foot,gross_rent
year,,,
2010,372560,369.344353,1239
2011,374507,341.903429,1530
2012,376454,399.389968,2324
2013,378401,483.600304,2971
2014,380348,556.277273,3528


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [26]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    plot1 = sfo_data2.plot(kind='bar',  xlabel='Year', 
               ylabel='Housing Units' , rot=90,  
               title='Housing Units in San Francisco from 2010 to 2016', 
               legend=False)
    
    return plot1

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    gross_rent = sfo_data.groupby(['year'])['gross_rent'].mean()

    plot2 = gross_rent.plot(kind='line',  xlabel='Year', 
               ylabel='Price per Sqft', color='red',   
               title='Average Gross Rent by Year', 
               legend=False)
    
    return plot2

def average_sales_price():
    """Average Sales Price Per Year."""
    
    sale_price_sqr_foot = sfo_data.groupby(['year'])['sale_price_sqr_foot'].mean()

    plot3 = sale_price_sqr_foot.plot(kind='line',  xlabel='Year', 
               ylabel='Price per Sqft', color='purple',   
               title='Average Price per Sqft by Year', 
               legend=False)
    
    return plot3



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    hood_mean = sfo_data.groupby(['year','neighborhood'])['sale_price_sqr_foot','gross_rent'].mean()

    avg_price = hood_mean.hvplot.line(x='year',xlabel='Year', y='sale_price_sqr_foot',
                      ylabel='Avg Sales Price per Square Foot', width=600, 
                         groupby='neighborhood')
    return avg_price
    
    
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    top_hood = sfo_data.groupby(['neighborhood'])['sale_price_sqr_foot', 'gross_rent','housing_units'].mean().nlargest(10, 'sale_price_sqr_foot').sort_values(by='sale_price_sqr_foot',ascending=False)


    plot4 = top_hood.hvplot.bar(x ='neighborhood',xlabel='neighborhood', y='sale_price_sqr_foot',
                      ylabel='Avg Sales Price per Square Foot', height=500, width=600, rot=90
                         ).sort(['sale_price_sqr_foot'],reverse=True)
    return plot4

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    hood_mean = sfo_data.groupby(['year','neighborhood'])['sale_price_sqr_foot','gross_rent','housing_units'].mean().sort_values(by='gross_rent',ascending=True)

    stack_hood = hood_mean.hvplot.bar(stacked=False,  x='year', xlabel='Neighborhood', y=['sale_price_sqr_foot','gross_rent'], 
                                  value_label='Num Housing Units', height=400, groupby='neighborhood', rot=90)
    return stack_hood


def neighborhood_map():
    """Neighborhood Map."""

    map_mean = sfo_data.groupby(['neighborhood'])['sale_price_sqr_foot','housing_units','gross_rent'].mean().reset_index()

    map_data = pd.concat([map_mean, neighborhood_data], axis='columns', join='inner')
    

    # Set the mapbox access token
    px.set_mapbox_access_token(map_box_api)

    # Create a scatter mapbox to analyze neighborhood info
    map_plot = px.scatter_mapbox(
        map_data,
        lat="Lat",
        lon="Lon",
         size="sale_price_sqr_foot",
         color="gross_rent",
          zoom=10
    )

    # Display the map
    return map_plot

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [27]:
# Create a Title for the Dashboard

yearly_column = pn.Column(
    housing_units_per_year(), average_gross_rent() + average_sales_price() 
)



hood_column = pn.Column(
    average_price_by_neighborhood(),
    top_most_expensive_neighborhoods(),
    most_expensive_neighborhoods_rent_sales()
)



map_column = pn.Column(
     neighborhood_map() 
)


# Create a tab layout for the dashboard
pop_dashboard = pn.Tabs(
    ("Welcome", map_column),
    ("Yearly Market Analysis", yearly_column),
    ("Neighborhood Analysis", hood_column)
)


# Create the dashboard
# YOUR CODE HERE!

## Serve the Panel Dashboard

In [28]:
# Serve the# dashboard
pop_dashboard.servable()

Tabs
    [0] Column
        [0] Plotly(Figure)
    [1] Column
        [0] HoloViews(Bars)
        [1] HoloViews(Layout)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [29]:
#housing_units_per_year()

In [30]:
#average_gross_rent()

In [31]:
# average_sales_price()

In [32]:
# average_price_by_neighborhood()

In [33]:
#top_most_expensive_neighborhoods()

In [34]:
#most_expensive_neighborhoods_rent_sales()

In [35]:
#neighborhood_map().show()

In [36]:
# parallel_categories()

In [37]:
# parallel_coordinates()

In [38]:
# sunburst()